In [1]:
import requests
import json
import pandas as pd
import numpy as np
import time
from tqdm import tqdm

In [2]:
API_KEY = "HDEV-d47d2366-856e-402a-97e3-9c648d1b6ec8"
headers = {
    "Authorization": API_KEY
}

In [56]:
url = "https://api.henrikdev.xyz/valorant/v1/by-puuid/stored-matches/eu/72d21e77-885b-5c73-8d98-6191655758a1"
response = requests.get(url, headers=headers)
response.json()

{'status': 200,
 'results': {'total': 76, 'returned': 76, 'before': 0, 'after': 76},
 'data': [{'meta': {'id': '5bb8c239-5e9d-4b60-b5c0-aeac4081a96c',
    'map': {'id': '2fe4ed3a-450a-948b-6d6b-e89a78e680a9', 'name': 'Lotus'},
    'version': 'release-11.00-shipping-14-3581174',
    'mode': 'Competitive',
    'started_at': '2025-06-28T13:52:47.393Z',
    'season': {'id': 'ac12e9b3-47e6-9599-8fa1-0bb473e5efc7',
     'short': 'e11aIV'},
    'region': 'eu',
    'cluster': 'Bahrain'},
   'stats': {'puuid': '72d21e77-885b-5c73-8d98-6191655758a1',
    'team': 'Red',
    'level': 191,
    'character': {'id': '1dbf2edd-4729-0984-3115-daa5eed44993',
     'name': 'Clove'},
    'tier': 19,
    'score': 6614,
    'kills': 24,
    'deaths': 18,
    'assists': 5,
    'shots': {'head': 16, 'body': 43, 'leg': 0},
    'damage': {'made': 4012, 'received': 3452}},
   'teams': {'red': 13, 'blue': 8}},
  {'meta': {'id': '5d6fcafa-0743-4a72-a4e4-6d2019a55bb3',
    'map': {'id': '92584fbe-486a-b1b2-9faa-39b0f

In [52]:
response.json().keys()

dict_keys(['status', 'results', 'data'])

In [46]:
df = pd.read_csv('matchs.csv')
list_puuid = []
for i in range(5):
    puuid = df[f'blue_{i}_puuid'].to_list()
    list_puuid += puuid
    puuid = df[f'red_{i}_puuid'].to_list()
    list_puuid += puuid

list_puuid = list(set(list_puuid))  # Remove duplicates
print(f"Total unique PUUIDs: {len(list_puuid)}")

Total unique PUUIDs: 1133


In [ ]:
def number_matches_retrieved(puuid, nb_retries=3):
    url = f"https://api.henrikdev.xyz/valorant/v1/by-puuid/stored-matches/eu/{puuid}"

    for attempt in range(nb_retries):
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()  # Raise an error for bad responses
            break  # Exit loop if request is successful
        except requests.exceptions.RequestException as e:
            tqdm.write(f"Attempt {attempt + 1} failed for PUUID {puuid}: {e}")
            if attempt == nb_retries - 1:
                raise
            time.sleep(60) # Wait before retrying
    data = response.json()
    if 'data' in data and isinstance(data['data'], list):
        return len(data['data'])

In [58]:
nb_matches = []
for puuid in tqdm(list_puuid[:50], desc="Retrieving number of matches", unit="PUUID"):
    try:
        number = number_matches_retrieved(puuid)
        nb_matches.append(number)
    except Exception as e:
        tqdm.write(f"Error retrieving data for PUUID {puuid}: {e}")

Retrieving number of matches:  30%|███       | 15/50 [00:09<00:19,  1.83PUUID/s]

Attempt 1 failed for PUUID ba98a6d9-8fde-5c5e-9f5e-07090916f347: 429 Client Error: Too Many Requests for url: https://api.henrikdev.xyz/valorant/v1/by-puuid/stored-matches/eu/ba98a6d9-8fde-5c5e-9f5e-07090916f347


Retrieving number of matches:  60%|██████    | 30/50 [01:17<00:13,  1.50PUUID/s]

Attempt 1 failed for PUUID 9e997a11-5e79-5cec-93ea-0a9dbe03de7e: 429 Client Error: Too Many Requests for url: https://api.henrikdev.xyz/valorant/v1/by-puuid/stored-matches/eu/9e997a11-5e79-5cec-93ea-0a9dbe03de7e


Retrieving number of matches:  60%|██████    | 30/50 [02:17<00:13,  1.50PUUID/s]

...

Retrieving number of matches:  62%|██████▏   | 31/50 [03:18<11:37, 36.69s/PUUID]

Attempt 3 failed for PUUID 9e997a11-5e79-5cec-93ea-0a9dbe03de7e: 404 Client Error: Not Found for url: https://api.henrikdev.xyz/valorant/v1/by-puuid/stored-matches/eu/9e997a11-5e79-5cec-93ea-0a9dbe03de7e
Error retrieving data for PUUID 9e997a11-5e79-5cec-93ea-0a9dbe03de7e: 404 Client Error: Not Found for url: https://api.henrikdev.xyz/valorant/v1/by-puuid/stored-matches/eu/9e997a11-5e79-5cec-93ea-0a9dbe03de7e


Retrieving number of matches:  92%|█████████▏| 46/50 [03:25<00:02,  1.64PUUID/s]

Attempt 1 failed for PUUID e1966e7e-d070-5b87-b90b-5e0b133dbf82: 429 Client Error: Too Many Requests for url: https://api.henrikdev.xyz/valorant/v1/by-puuid/stored-matches/eu/e1966e7e-d070-5b87-b90b-5e0b133dbf82


Retrieving number of matches: 100%|██████████| 50/50 [04:27<00:00,  5.35s/PUUID]


In [3]:
def retrieve_matches(puuid, nb_retries=3):
    url = f"https://api.henrikdev.xyz/valorant/v1/by-puuid/stored-matches/eu/{puuid}"

    for attempt in range(nb_retries):
        try:
            response = requests.get(url, headers=headers, params={"mode":'competitive'})
            response.raise_for_status()  # Raise an error for bad responses
            break  # Exit loop if request is successful
        except requests.exceptions.RequestException as e:
            tqdm.write(f"Attempt {attempt + 1} failed for PUUID {puuid}: {e}")
            if attempt == nb_retries - 1:
                raise
            time.sleep(60) # Wait before retrying
    data = response.json()
    return data

In [ ]:
import pandas as pd
from tqdm import tqdm

def stats_player(matches_list):
    def extract(match):
        stats = match.get('stats')
        if not stats:
            tqdm.write(f"No stats found for match with PUUID {match.get('puuid', 'UNKNOWN')}")
            return None

        shots = stats.get('shots', {})
        damage = stats.get('damage', {})

        return {
            'level': stats.get('level'),
            'tier': stats.get('tier'),
            'score': stats.get('score'),
            'kills': stats.get('kills'),
            'deaths': stats.get('deaths'),
            'assists': stats.get('assists'),
            'headshots': shots.get('head'),
            'bodyshots': shots.get('body'),
            'legshots': shots.get('leg'),
            'damage_made': damage.get('made'),
            'damage_received': damage.get('received'),
        }

    extracted = filter(None, (extract(match) for match in tqdm(matches_list, desc="Processing matches", unit="match")))
    stats_df = pd.DataFrame(extracted)

    
    mean_stats = stats_df.drop(columns='level', errors='ignore').mean(numeric_only=True).to_dict()
    mean_stats['level'] = int(stats_df['level'].max()) if 'level' in stats_df else None
    mean_stats['puuid'] = matches_list[0].get('stats',{}).get('puuid', 'UNKNOWN')
    return mean_stats


In [52]:
stats = stats_player(retrieve_matches("72d21e77-885b-5c73-8d98-6191655758a1")['data'])

Processing matches: 100%|██████████| 78/78 [00:00<?, ?match/s]


In [57]:
stats

{'tier': 19.05128205128205,
 'score': 5481.666666666667,
 'kills': 18.794871794871796,
 'deaths': 16.82051282051282,
 'assists': 5.448717948717949,
 'headshots': 14.08974358974359,
 'bodyshots': 41.51282051282051,
 'legshots': 3.1025641025641026,
 'damage_made': 3604.551282051282,
 'damage_received': 3358.0128205128203,
 'level': np.int64(192),
 'puuid': '72d21e77-885b-5c73-8d98-6191655758a1'}